# SpaceX Launch Site Geospatial Analysis

## Project Overview

The success of a SpaceX launch is influenced by multiple factors such as payload mass, orbit type, and mission profile. In addition to these technical parameters, **geographical location and surrounding infrastructure** play a crucial role in launch feasibility and operational safety.

This notebook performs a **geospatial analysis of SpaceX launch sites** using interactive maps created with **Folium**. The analysis focuses on understanding the spatial distribution of launch sites and their proximity to critical geographical features such as coastlines, cities, railways, and highways.

## Objectives


The objective of this analysis is to:
- Visualize SpaceX launch site locations on an interactive map
- Analyze launch outcomes (successful vs failed) by location
- Compute distances between launch sites and nearby infrastructure
- Identify spatial patterns that may influence launch success

## Tools & Libraries
- **Pandas** – data manipulation
- **Folium** – interactive geospatial visualization
- **Folium Plugins** – MarkerCluster, MousePosition, DivIcon

In [22]:
import folium
import pandas as pd

In [23]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Launch Site Visualization


In this section, we visualize the geographical locations of all SpaceX launch sites using their latitude and longitude coordinates. This provides a spatial overview of launch infrastructure distribution.


The dataset `spacex_launch_geo.csv` contains geographical coordinates for each SpaceX launch site and has been augmented during earlier data preparation steps to support spatial analysis.


In [24]:
spacex_df=pd.read_csv('spacex_launch_geo.csv')

Now, you can take a look at what are the coordinates for each site.


In [25]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Raw latitude and longitude values provide limited intuitive understanding of spatial relationships. To better interpret the geographical distribution of SpaceX launch sites, we visualize these coordinates on an interactive map.

The map is initialized with a central reference point to provide geographic context before plotting individual launch sites.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [26]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate: 


In [27]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


## Mapping SpaceX Launch Sites


In [28]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle + label
for _, row in launch_sites_df.iterrows():

    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']

    folium.Circle(
        location=coordinate,
        radius=1000,
        color='#d35400',
        fill=True,
        fill_opacity=0.5
    ).add_child(folium.Popup(site_name)).add_to(site_map)

    folium.Marker(
        location=coordinate,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#353935;"><b>%s</b></div>' % 'NASA JSC',
        )
    ).add_to(site_map)

# DISPLAY MAP (required)
site_map

## Launch Outcome Visualization


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [29]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [30]:
marker_cluster = MarkerCluster()


In [31]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [32]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [33]:
# Add marker_cluster to the existing site_map (do this ONCE)
site_map.add_child(marker_cluster)

# For each launch result in spacex_df, add a marker to the cluster
for _, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(
            icon='info-sign',
            color=record['marker_color']  # green if success, red if failure
        ),
        popup=(
            f"Launch Site: {record['Launch Site']}<br>"
            f"Success: {record['class']}"
        )
    ).add_to(marker_cluster)

# Display map
site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


## Proximity Analysis of Launch Sites


In this section, the spatial proximity of SpaceX launch sites to surrounding geographical features is analyzed. Understanding these distances provides insights into logistical, safety, and operational considerations influencing launch site selection like railways,sea coast etc.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [34]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

The distance between a launch site and surrounding geographical features is computed using the Haversine formula, which calculates great-circle distances between two points on the Earth based on latitude and longitude.


### Distance Calculation Methodology


In [35]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

### Distance to Nearest Coastline


To evaluate safety and operational feasibility, the distance between the launch site and the nearest coastline is calculated. Launch sites are typically positioned close to coastlines to allow safe downrange trajectories over open water during ascent.


In [36]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Launch site coordinate (example: CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Closest coastline coordinate (picked using MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance between launch site and coastline
distance_coastline = calculate_distance(
    launch_site_lat,
    launch_site_lon,
    coastline_lat,
    coastline_lon
)

distance_coastline

0.5097431144955059

In [37]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# Coordinate of the selected closest coastline point
coastline_coordinate = [coastline_lat, coastline_lon]

# Create a marker to show the distance
distance_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(150, 20),
        icon_anchor=(0, 0),
        html=f'''
        <div style="font-size:12px; color:#353935; font-weight:bold;">
            {distance_coastline:.2f} KM
        </div>
        '''
    )
)

# Add marker to the map
site_map.add_child(distance_marker)
##353935
site_map

# Create a PolyLine


A line is drawn between the launch site and the nearest coastline point to visually represent the calculated distance on the map.


In [38]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# Coordinates for the polyline
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

# Create PolyLine
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,
    color='blue'
)

# Add to map
site_map.add_child(lines)

# Display map
site_map

# Create a marker with distance to a closest city, railway, highway, etc.


### Proximity to Transportation Infrastructure and Cities


A railway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In addition to coastline proximity, distances to nearby railways, highways, and urban centers are calculated. These measurements provide insight into logistical accessibility while ensuring adequate separation from densely populated regions.


In [39]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Launch site: CCAFS SLC-40
launch_site_lat = 28.563197
launch_site_lon = -80.576820
launch_site_coord = [launch_site_lat, launch_site_lon]


# Railway (NASA Railroad) — CONFIRMED
railway_lat = 28.52393
railway_lon = -80.78957
railway_coord = [railway_lat, railway_lon]

# Highway (US-1 / NASA Causeway) — CONFIRMED
highway_lat = 28.52393
highway_lon = -80.78957
highway_coord = [highway_lat, highway_lon]

# City (Orlando – city center)
city_lat = 28.538336
city_lon = -81.379234
city_coord = [city_lat, city_lon]

#distances
#railway
railway_dist = calculate_distance(
    launch_site_lat, launch_site_lon,
    railway_lat, railway_lon
)
#city
city_dist = calculate_distance(
    launch_site_lat, launch_site_lon,
    city_lat, city_lon
)

In [40]:
from folium.features import DivIcon

def draw_distance(site_map, start, end, distance, label, color):
    # Line
    folium.PolyLine(
        locations=[start, end],
        color=color,
        weight=2
    ).add_to(site_map)

    # Label at destination
    folium.Marker(
        location=end,
        icon=DivIcon(
            icon_size=(220, 20),
            icon_anchor=(0, 0),
            html=f'''
            <div style="font-size:12px; color:{color}; font-weight:bold;">
                {distance:.2f} KM ({label})
            </div>
            '''
        )
    ).add_to(site_map)

In [41]:
# Railway
draw_distance(
    site_map,
    launch_site_coord,
    railway_coord,
    railway_dist,
    "Railway",
    "green"
)
# City
draw_distance(
    site_map,
    launch_site_coord,
    city_coord,
    city_dist,
    "City (Orlando)",
    "blue"
)
site_map

### Key Observations

The proximity analysis reveals that SpaceX launch sites are strategically positioned close to coastlines and key transportation infrastructure such as railways and highways, enabling efficient logistics and transportation of launch components.

At the same time, launch sites maintain a measurable distance from major urban centers, reducing risks to populated areas during launch operations. This balance between accessibility and safety reflects deliberate planning in launch site selection.
